In [1]:
import sys
sys.path.append("..")
from src.environment import *
from src.elements import *
from src.dynamics import *
from utils.test_utils import *
from src.methods import *

In [2]:
dynamics = D2Q9()
key = Key(1234)
Element.dynamics = dynamics
Environment.dynamics = dynamics
n_cells = 1000
n_faces = 1000
n_nodes = 1000
cells = [Cell.pdf_init(jax.random.uniform(key(),dynamics.ones_pdf().shape),jnp.asarray([])) for i in range(n_cells)]
faces = [Face.pdf_init(jax.random.uniform(key(),dynamics.ones_pdf().shape),jnp.asarray([]),jnp.asarray([]),jnp.asarray([])) for i in range(n_faces)]
nodes = [Node.pdf_init(jax.random.uniform(key(),dynamics.ones_pdf().shape),jax.random.randint(key(),(),0,2),jnp.asarray([]),jnp.asarray([])) for i in range(100)]
for node in nodes:
    n = jax.random.randint(key(),(),2,10)
    node.type = jnp.asarray(1)
    node.cell_dists = jnp.abs(jax.random.normal(key(),(n,)))
    node.cells_index = jax.random.randint(key(),(n,),0,n_cells)
env = Environment.create(cells,faces,nodes)
env.setMethods(Methods)

In [3]:
params,config = env.init()
jax.tree.structure(params)

PyTreeDef({'cells': {'pdf': *, 'pdf_eq': *, 'rho': *, 'vel': *}, 'faces': {'flux': *, 'pdf': *, 'rho': *, 'vel': *}, 'nodes': {'Velocity': {'pdf': *, 'rho': *, 'vel': *}}})

Cell Eq Test

In [ ]:
jitted = jax.jit(env.methods.calc_cell_eqs)
jitted(params)
%timeit -n100 -r10 env.methods.calc_cell_eqs(params)
%timeit -n100 -r10 jitted(params)

Cell Macros Test

In [ ]:
jitted = jax.jit(env.methods.calc_cell_macros)
jitted(params)
%timeit -n100 -r10 env.methods.calc_cell_macros(params)
%timeit -n100 -r10 jitted(params)

1.36 ms ± 14.4 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
119 μs ± 2.93 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [ ]:
jitted = jax.jit(env.methods.calc_node_pdfs)
print(params["nodes"]["Velocity"]["pdf"][0])
params = jitted(params,config)
print(params["nodes"]["Velocity"]["pdf"][0])
%timeit -n100 -r10 env.methods.calc_node_pdfs(params,config)
%timeit -n100 -r10 jitted(params,config)

[0.6457553  0.06957352 0.41975212 0.9838723  0.08163345 0.3375119
 0.3994317  0.3936324  0.7771797 ]
[-0.01713668 -0.0027834  -0.00398572 -0.00661427 -0.00460463 -0.00065486
 -0.00154562 -0.0017673  -0.00074262]
5.66 ms ± 699 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
39.9 μs ± 1.74 μs per loop (mean ± std. dev. of 10 runs, 100 loops each)
